"""classical_models.ipynb
by: Archie Gertsman (arkadiy2@illinois.edu)
Project director: Richard Sowers
r-sowers@illinois.eduhttps://publish.illinois.edu/r-sowers/
Copyright 2019 University of Illinois Board of Trustees. All Rights Reserved. Licensed under the MIT license
"""

In [1]:
import sys
sys.path.append('../src/')
sys.path.append('../data/')
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from modeling_helpers import *

In [8]:
df = pd.read_pickle('../data/block4_updated.pkl')

agg_dict={
    'xtrack_dist': ['std'],
    'avg_surr_speed': ['mean','std'],
    'lanes':['mean'],
    'len':['mean'],
    'speed':['mean','std'],
    'vehicle_density': ['mean'],
    'lon_acc': ['mean','std'],
    'lat_acc': ['mean','std']
}

df = df[list(agg_dict.keys()) + ['type']]
df.index = [df.index.map(lambda idx: f'{idx[0]}_{idx[1]}'),  
            df.index.get_level_values(2)]
df.index.names = ['id','road']
df

xtrack_dist  avg_surr_speed  lanes      len  \
id       road                                                                
4_1_1    250699362_250699984     -1.883401       10.464171    5.4   97.581   
         250699362_250699984     -1.980795       10.457843    5.4   97.581   
         250699362_250699984     -1.937041       10.452857    5.4   97.581   
         250699362_250699984     -1.893287       10.448586    5.4   97.581   
         250699362_250699984     -1.941984       10.444986    5.4   97.581   
...                                    ...             ...    ...      ...   
1_3_2116 300400248_8446047162     0.658316       26.005850    6.5  106.837   
         300400248_8446047162     0.764478       26.008150    6.5  106.837   
         300400248_8446047162     0.736857       26.010500    6.5  106.837   
         300400248_8446047162     0.783032       26.012950    6.5  106.837   
         300400248_8446047162     0.755411       26.015600    6.5  106.837   

                                 speed  vehicle_density  lon_acc  lat_acc  \
id       road                                                               
4_1_1    250699362_250699984   11.9046                7  -0.1145   0.0138   
         250699362_250699984   11.8975                7  -0.1007   0.0147   
         250699362_250699984   11.8919                7  -0.0918   0.0157   
         250699362_250699984   11.8871                7  -0.0869   0.0167   
         250699362_250699984   11.8831                7  -0.0784   0.0176   
...                                ...              ...      ...      ...   
1_3_2116 300400248_8446047162  25.3482                2  -0.3011   0.3234   
         300400248_8446047162  25.3217                2  -0.3032   0.3216   
         300400248_8446047162  25.2946                2  -0.3012   0.3180   
         300400248_8446047162  25.2667                2  -0.3054   0.3124   
         300400248_8446047162  25.2370                2  -0.3135   0.3045   

                               type  
id       road                        
4_1_1    250699362_250699984   Taxi  
         250699362_250699984   Taxi  
         250699362_250699984   Taxi  
         250699362_250699984   Taxi  
         250699362_250699984   Taxi  
...                             ...  
1_3_2116 300400248_8446047162   Car  
         300400248_8446047162   Car  
         300400248_8446047162   Car  
         300400248_8446047162   Car  
         300400248_8446047162   Car  

[2997687 rows x 9 columns]

In [11]:
# min_traj_len = 300
# min_speed_ratio = 1

# def speed_ratio(grp, min_speed=0):
#     return len(grp[grp.speed > min_speed]) / len(grp)

# df = df.groupby(['id','road']) \
#     .filter(lambda grp: (len(grp) >= min_traj_len) & (speed_ratio(grp) >= min_speed_ratio))
# df

In [46]:
df_agg = downsample(df, 100, 0.3, agg_dict)
df_agg_train, df_agg_test = train_test_split_vehicles(df_agg, 200)

In [ ]:
model = GradientBoostingClassifier()
X_train,y_train = split_X_y(df_agg_train)
model.fit(X_train, y_train)

X_test,y_test = split_X_y(df_agg_test)
accuracy(model, X_test, y_test)